---
- SLNC의 2025.01월분 네이버 고객리뷰 데이터를, GPT API를 사용하여 감성분석 
- ref. https://colab.research.google.com/drive/1ord7ND2l0lvAhHyIfAMinxAOYBeUUlyj?usp=sharing#scrollTo=fcfcg517BGSa 
---

In [ ]:
#!pip install --upgrade openai
import pandas as pd
import time
import openai
import xlwings as xw
from tqdm import tqdm
import os
from openai.error import OpenAIError

# OpenAI 클라이언트 초기화 
openai.api_key = "**********"

# 리뷰 데이터 로딩 (xlwings 사용)
book = xw.Book('data/reviews_20250228.xlsx')
sheet = book.sheets[0]
df = sheet.used_range.options(pd.DataFrame, index=False).value  
print(df.shape)

def handle_api_error(e):
    """
    OpenAI API 오류를 처리하는 함수
    """
    error_message = str(e).lower()

    if "rate limit" in error_message:
        retry_time = 30  # 기본 재시도 시간 30초
        print(f"💡 Rate limit exceeded. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)

    elif "service unavailable" in error_message:
        retry_time = 10  # 서버 오류 시 10초 후 재시도
        print("💡 OpenAI Service is unavailable. Retrying in 10 seconds...")
        time.sleep(retry_time)

    elif "timeout" in error_message or "connection error" in error_message:
        retry_time = 15  # 네트워크 오류 시 15초 후 재시도
        print(f"💡 API Connection/Timeout error: {e}. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)

    else:
        print(f"OpenAI API 에러 발생: {e}")

def analyze_review(review, max_retries=3):
    """
    고객 리뷰를 분석하여 긍정/부정을 판단하는 함수
    """
    messages = [
        {"role": "system", "content": "너는 고객 리뷰를 감정 분석하는 AI 모델이야."},
        {"role": "user", "content": f"다음 리뷰를 분석해 '긍정' 또는 '부정' 둘 중 하나로 답해: {review}"}
    ]

    retry_count = 0
    while retry_count < max_retries:
        try:
            completion = openai.ChatCompletion.create(
                model="gpt-4-turbo",
                messages=messages,
                max_tokens=3,
                temperature=0
            )
            return completion["choices"][0]["message"]["content"].strip()
        except OpenAIError as e:
            retry_count += 1
            handle_api_error(e)
    
    return "오류"

def analyze_procon(review):
    """
    고객 리뷰에서 긍정 단어와 부정 단어를 추출하는 함수
    """
    messages = [
        {"role": "system", "content": "너는 리뷰에서 긍정 단어와 부정 단어를 추출하는 AI 모델이야."},
        {"role": "user", "content": f"다음 리뷰에서 긍정 단어와 부정 단어를 나열해줘: {review}"}
    ]
    
    try:
        completion = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=messages,
            max_tokens=50,
            temperature=0
        )
        response = completion["choices"][0]["message"]["content"].strip()
        
        # 긍정 단어와 부정 단어 분리
        if "\n" in response:
            positive_words, negative_words = response.split("\n", 1)
        elif "," in response:
            positive_words, negative_words = response.split(",", 1)
        else:
            positive_words, negative_words = response, "없음"

        return positive_words.strip(), negative_words.strip()

    except OpenAIError as e:
        return "없음", "없음"

# 감정 분석 및 주요 단어 추출 진행
df["Sentiment"] = df["content"].fillna("").apply(analyze_review)
df[["Positive_Words", "Negative_Words"]] = df["content"].fillna("").apply(lambda x: pd.Series(analyze_procon(x)))

# 엑셀 파일로 저장
output_path = "output/sentiment_analysis_new.xlsx"
df.to_excel(output_path, index=False)
print(f"감정 분석 결과가 저장되었습니다: {output_path}")  
